# Learn Lyncs-API

In [ ]:
import os
import sys
import numpy as np  # type: ignore
import lyncs_io as lio  # type: ignore
from typing import List

<div class="alert alert-block alert-warning"><b>Warning:</b>If you get the warning <code><FONT COLOR="#db5048">ModuleNotFoundError</code><code>: No module named 'lyncs_io'</code>, you need to change the kernel via Kernel > Change kernel > lyncs.</div>

In [ ]:
import plaquette as RPlaq

## Introduction

<a href="https://github.com/Lyncs-API">Lyncs-API</a> is a Python API for Lattice QCD applications () created by Simone Bacchio, Christodoulos Stylianou and Alexandros Angeli.

One such API is <a href="https://github.com/Lyncs-API/lyncs.io">lyncs.io</a>, a suite of I/O functions which allow for quick and simple interfacing with several common file formats, including lattice-oriented formats such as `.lime` and `.openqcd`.

## Accessing the data

Let us use some example gaugefield files to showcase `lyncs`. We use $8\times24^3$ SU(3) $N_f = 2 + 1$ gaugefields in $3+1d$. These otherwise have the same parameters as FASTSUM's Generation 2 ensembles \[[1](https://arxiv.org/abs/1412.6411), [2](https://arxiv.org/abs/2007.04188)\].

In [ ]:
# the location, name and ID number of the gaugefields
gfDir = 'confs'
gfName = 'Gen2_8x24n'
gfIDs = [7,8,9]

Using the `head()` function, we can read the header data stored in the file and access the information in the form of a `dict` object.

Here we see that our example gaugefield files are arranged in the shape $N_t \times N_s^3 \times N_d \times N_c^2$. The `dtype` key shows that the datatype is `'<c16'` or (little-endian) double-precision complex. `'_offset'` is the number of bytes in the header and `'plaq'` is the average value of the spatial and temporal plaquette.

In [ ]:
# We can probe the header data of the gaugefield files
# Loop over each ID
for iid in gfIDs:
    gfFile = os.path.join(gfDir,f'{gfName}{iid}')
    # Read and print header
    print(f"{gfFile}:", lio.head(gfFile, format='openqcd'))

The `load()` function returns a `numpy.ndarray` containing the gaugefields. Let's load some example data:

In [ ]:
# Load the gaugefields
# Make a list of gaugefield data
gfData: List[np.ndarray] = []
# Loop over each ID
for iid in gfIDs:
    gfFile = os.path.join(gfDir,f'{gfName}{iid}')
    # Load and append
    # Here we have specified the full path and the format
    # Can figure it out based on extension, but format is clearer
    gfData.append(lio.load(gfFile, format='openqcd'))
# Convert to array for better indexing
gfAr = np.asarray(gfData)

`lyncs.io` can also be used to convert from one format to another. For example, we can simply convert from `openqcd` format to `lime` using the `save()` function:

In [ ]:
# Save the gaugefield array gfAr to a lime file
lio.save(gfAr,'Gen2_8x24_gfAr.lime')

Since our new file has a standard extension, `lyncs.io` can infer the format from the filename. The `head()` function now accesses the `lime` record associated with our new data.

In [ ]:
# Read the header of our new lime file
lio.head('Gen2_8x24_gfAr.lime')

If we want to access all of the records in the file, we can use `lime.read_records()`:

In [ ]:
lio.lime.read_records('Gen2_8x24_gfAr.lime')

If we want to read this lime file back into a `numpy.ndarray` we can use the `load()` function again

In [ ]:
lime = lio.load('Gen2_8x24_gfAr.lime')

## Manipulating the data

As an exercise, let's calculate the values for the whole, spatial and temporal plaquettes:
<img src="latticePlaqDiag.png" alt="plaquette diagram" width="400" height="400"/>

In [ ]:
# Here show the shape of the data
# it is [ID, NT, NX, NY, NZ, mu, colour, colour]
print(gfAr.shape)

In [ ]:
# Use Ryan's plaquette code to calculate whole of lattice plaquette
# the sum of plaquettes, the number of plaquttes, the average plaquette and the time taken
sumTrP, nP, ave, time = RPlaq.plaquette(gfAr[0, ...])
print(f'calculated average plaquette {ave} in {time:.2} seconds')

In [ ]:
# Use Ryan's plaquette code to calculate spatial plaquette
ssumTrP, snP, save, stime = RPlaq.plaquette(gfAr[0, ...], muStart=1, muEnd=4, nuEnd=4)
print(f'calculated average spatial plaquette {save} in {stime:.2} seconds')

In [ ]:
# Use Ryan's plaquette code to calculate temporal plaquette
tsumTrP, tnP, tave, ttime = RPlaq.plaquette(gfAr[0, ...], muStart=0, muEnd=1, nuEnd=4)
print(f'calculated average temporal plaquette {tave} in {ttime:.2} seconds')

In [ ]:
# Just check that this agrees with the whole of lattice plaquette (visually)
print(f'average of temporal and spatial plaquettes {(tave+save)/2.0}')

# C

Now that we can load and manipulate the data, we might want to increase performance by offloading our calculations to a script written in a compiled language such as C.

In [ ]:
# Prepare a single gaugefield for writing
COrder = gfAr[0, ...]
# Now save
COrder.tofile('Gen2_8x24_gfAr0.C')

There is a C program `readC.c` supplied. This program will read `Gen2_8x24_gfAr0.C` in and calculate whole, spatial and temporal plaquettes. We do that now

In [ ]:
# The command to compile
ccmd = 'gcc -O3 readC.c'
os.system(ccmd)

In [ ]:
# The command to run
rcmd = './a.out'
os.system(rcmd)

# Fortran
We might like to read the gaugefield into Fortran too. While there exists code that read openqcd format into Fortran, these do not undo the checkerboarding or `flattening` of the space-time dimensions. Here we seek to have the data in the same $N_t \times N_s^3 \times N_d \times N_c^2$ format as in the `np.ndarray`.

Unlike C which uses row-major ordering, Fortran uses column-major ordering for multidimensional arrays in linear memory. We must specify this reordering when calling `reshape` by passing the argument `order='F'`.

<div class="alert alert-block alert-info"><b>Info:</b> Python natively uses neither row nor column ordering. Instead, the allocations are made directly onto the heap (which is not necessarily contiguous) rather than the stack. However, the <code>numpy</code> package is based in C and thus follows the row-major ordering scheme.</div>

This solution is based upon [this stack overflow](https://stackoverflow.com/a/49179272) answer. This solution assumes that your Python and Fortran code use the same (little) endianess.

In [ ]:
# First reorder a single gaugefield into 'fortran' order
fortOrder = gfAr[0, ...].reshape(gfAr[0, ...].shape, order='F')
# we consider a single gaugefield only as Fortran allows only rank 7 arrays 
# and saving all configurations at once would be rank 8
# This could be avoided by some sort of derived type
# Now we save
fortOrder.T.tofile('Gen2_8x24_gfAr0.fort')

There is a Fortran program `readFortran.f90` supplied. This program will read `Gen2_8x24_gfAr0.fort` in and calculate whole, spatial and temporal plaquettes. We do that now

In [ ]:
# The command to compile
ccmd = 'gfortran -O3 -g -fbacktrace readFortran.f90'
os.system(ccmd)

In [ ]:
# The command to run
rcmd = './a.out'
os.system(rcmd)

### 